In [9]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, timezone

In [6]:
# Configuration des APIs
AQICN_API_KEY = "d6dc128a0628a9274ab78ee69152ca8d572dc7cc"
OPENWEATHER_API_KEY = "dd2d8c2c92a3ebe773b453884a7f08c0"

# Villes françaises avec coordonnées
FRENCH_CITIES = [
    ("Paris", 48.8566, 2.3522),
    ("Marseille", 43.2965, 5.3698),
    ("Lyon", 45.7640, 4.8357),
    ("Toulouse", 43.6045, 1.4440),
    ("Nice", 43.7102, 7.2620),
    ("Nantes", 47.2184, -1.5536),
    ("Montpellier", 43.6109, 3.8772),
    ("Strasbourg", 48.5734, 7.7521),
    ("Bordeaux", 44.8378, -0.5792),
    ("Lille", 50.6292, 3.0573),
    ("Rennes", 48.1173, -1.6778),
    ("Reims", 49.2583, 4.0317),
    ("Saint-Étienne", 45.4397, 4.3872),
    ("Le Havre", 49.4944, 0.1079),
    ("Toulon", 43.1242, 5.9280),
    ("Grenoble", 45.1885, 5.7245),
    ("Dijon", 47.3220, 5.0415),
    ("Angers", 47.4784, -0.5632),
    ("Nîmes", 43.8367, 4.3601),
    ("Villeurbanne", 45.7719, 4.8902)
]

In [3]:
len(FRENCH_CITIES)

20

OPENWEATHER

In [ ]:
#test appel API
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={key}&units=metric&lang=fr"

In [11]:
# ⚙️ Paramètres
lat, lon = 48.8566, 2.3522  # Paris
units = "metric"
lang = "fr"

# 🔗 URL One Call 3.0
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=air_pollution&units={units}&lang={lang}&appid={OPENWEATHER_API_KEY}"


response = requests.get(url)
data = response.json()

# Vérification du timezone_offset
tz_offset = data.get('timezone_offset', 0)  # si absent, on prend 0 (UTC)

def convert_timestamp(ts, tz_offset=0):
    return datetime.fromtimestamp(ts + tz_offset, tz=timezone.utc)

# Météo actuelle
current = data.get('current', {})
if current:
    current_df = pd.DataFrame([{
        'datetime': convert_timestamp(current.get('dt', 0), tz_offset),
        'temp': current.get('temp'),
        'feels_like': current.get('feels_like'),
        'humidity': current.get('humidity'),
        'pressure': current.get('pressure'),
        'clouds': current.get('clouds'),
        'wind_speed': current.get('wind_speed'),
        'wind_deg': current.get('wind_deg'),
        'description': current.get('weather', [{}])[0].get('description')
    }])
else:
    current_df = pd.DataFrame()  # DataFrame vide si current absent

# Prévisions quotidiennes
daily_list = []
for day in data.get('daily', []):
    daily_list.append({
        'date': convert_timestamp(day.get('dt',0), tz_offset).date(),
        'temp_min': day.get('temp', {}).get('min'),
        'temp_max': day.get('temp', {}).get('max'),
        'humidity': day.get('humidity'),
        'pressure': day.get('pressure'),
        'clouds': day.get('clouds'),
        'wind_speed': day.get('wind_speed'),
        'wind_deg': day.get('wind_deg'),
        'description': day.get('weather', [{}])[0].get('description')
    })

daily_df = pd.DataFrame(daily_list)

# Affichage
print("=== Météo actuelle ===")
print(current_df)
print("\n=== Prévisions quotidiennes ===")
print(daily_df)


=== Météo actuelle ===
Empty DataFrame
Columns: []
Index: []

=== Prévisions quotidiennes ===
Empty DataFrame
Columns: []
Index: []
